In [ ]:
'''
    @Author: King
    @Date: 2019.03.19
    @Purpose: 中文文本分类实战（六）—— RCNN模型
    @Introduction:  下面介绍
    @Datasets: ChnSentiCorp 中文情感分析数据集，其目标是判断一段话的情感态度。
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/RCNN
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208290.html
'''

## 数据集

ChnSentiCorp 中文情感分析数据集，其目标是判断一段话的情感态度，总共有三个数据文件，在/data/ChnSentiCorp_cn_data目录下，包括ChnSentiCorp_htl_all.csv。在进行文本分类时需要有标签的数据（labeledTrainData），数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为/data/ChnSentiCorp_htl_all.csv/labeledTrain.csv。


## RCNN 模型结构

RCNN模型来源于论文Recurrent Convolutional Neural Networks for Text Classification {https://arxiv.org/abs/1609.04243}。模型结构图如下：

![avatar](../images/RCNN.png)

　RCNN 整体的模型构建流程如下：

　　1）利用 Bi-LSTM 获得上下文的信息，类似于语言模型。

　　2）将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput, wordEmbedding, bwOutput]。

　　3）将拼接后的向量非线性映射到低维。

　　4）向量中的每一个位置的值都取所有时序上的最大值，得到最终的特征向量，该过程类似于max-pool。

　　5）softmax分类。


In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    outputSize = 128  # 从高维映射到低维的神经元个数
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/ChnSentiCorp_cn_data/labeledCharTrain.csv"
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [10]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["label"].tolist()
        review = df["review_cut_word"].tolist()
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r", encoding="utf-8") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [11]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (4612, 200)
train label shape: (4612, 1)
eval data shape: (1154, 200)


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [12]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [13]:
"""
构建模型，模型的架构如下：
1，利用Bi-LSTM获得上下文的信息
2，将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput;wordEmbedding;bwOutput]
3，将2所得的词表示映射到低维
4，hidden_size上每个位置的值都取时间步上最大的值，类似于max-pool
5，softmax分类
"""

class RCNN(object):
    """
    RCNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            # 复制一份embedding input
            self.embeddedWords_ = self.embeddedWords
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords_, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords_ = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        fwOutput, bwOutput = tf.split(self.embeddedWords_, 2, -1)
            
        with tf.name_scope("context"):
            shape = [tf.shape(fwOutput)[0], 1, tf.shape(fwOutput)[2]]
            self.contextLeft = tf.concat([tf.zeros(shape), fwOutput[:, :-1]], axis=1, name="contextLeft")
            self.contextRight = tf.concat([bwOutput[:, 1:], tf.zeros(shape)], axis=1, name="contextRight")
            
        # 将前向，后向的输出和最早的词向量拼接在一起得到最终的词表征
        with tf.name_scope("wordRepresentation"):
            self.wordRepre = tf.concat([self.contextLeft, self.embeddedWords, self.contextRight], axis=2)
            wordSize = config.model.hiddenSizes[-1] * 2 + config.model.embeddingSize 
        
        with tf.name_scope("textRepresentation"):
            outputSize = config.model.outputSize
            textW = tf.Variable(tf.random_uniform([wordSize, outputSize], -1.0, 1.0), name="W2")
            textB = tf.Variable(tf.constant(0.1, shape=[outputSize]), name="b2")
            
            # tf.einsum可以指定维度的消除运算
            self.textRepre = tf.tanh(tf.einsum('aij,jk->aik', self.wordRepre, textW) + textB)
            
        # 做max-pool的操作，将时间步的维度消失
        output = tf.reduce_max(self.textRepre, axis=1)
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [19]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = 0
    try:
        auc = roc_auc_score(trueY, predY)
    except ValueError:
        pass
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [21]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = RCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/RCNN/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/RCNN/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-03-19T10:27:10.129413, step: 45, loss: 0.3901325762271881, acc: 0.8594, auc: 0.7157, precision: 0.0, recall: 0.0
2019-03-19T10:27:11.924993, step: 46, loss: 0.4274086058139801, acc: 0.8438, auc: 0.5977, precision: 0.0, recall: 0.0
2019-03-19T10:27:13.792839, step: 47, loss: 0.3555102050304413, acc: 0.8984, auc: 0.5244, precision: 0.0, recall: 0.0
2019-03-19T10:27:15.607535, step: 48, loss: 0.41747844219207764, acc: 0.8516, auc: 0.5587, precision: 0.0, recall: 0.0
2019-03-19T10:27:17.359015, step: 49, loss: 0.43100255727767944, acc: 0.8438, auc: 0.5648, precision: 0.0, recall: 0.0
2019-03-19T10:27:19.174771, step: 50, loss: 0.3271244764328003, acc: 0.8906, auc: 0.7845, precision: 0.0, recall: 0.0

Evaluation:
2019-03-19T10:27:35.727478, step: 50, loss: 3.1832034587860107, acc: 0.0, auc: 0.0, precision: 0.0, recall: 0.0
Saved model checkpoint to ../model/RCNN/model/my-model-50

2019-03-19T10:27:38.324429, step: 51, loss: 0.40068385004997253, acc: 0.8516, auc: 0.6721, precision: 0.0,

2019-03-19T10:30:42.203761, step: 105, loss: 0.2162470668554306, acc: 0.9219, auc: 0.9619, precision: 0.0, recall: 0.0
2019-03-19T10:30:44.084604, step: 106, loss: 0.3345164656639099, acc: 0.875, auc: 0.7573, precision: 0.0, recall: 0.0
2019-03-19T10:30:45.952552, step: 107, loss: 0.36391714215278625, acc: 0.8438, auc: 0.8319, precision: 0.0, recall: 0.0
2019-03-19T10:30:47.713759, step: 108, loss: 0.3737369477748871, acc: 0.8359, auc: 0.8144, precision: 0.0, recall: 0.0
start training model
2019-03-19T10:30:49.583470, step: 109, loss: 0.4084371328353882, acc: 0.8359, auc: 0.7081, precision: 0.0, recall: 0.0
2019-03-19T10:30:51.532864, step: 110, loss: 0.3288953900337219, acc: 0.8672, auc: 0.8601, precision: 0.0, recall: 0.0

Evaluation:
2019-03-19T10:31:07.854030, step: 110, loss: 2.612167755762736, acc: 0.0, auc: 0.0, precision: 0.0, recall: 0.0
Saved model checkpoint to ../model/RCNN/model/my-model-110

2019-03-19T10:31:10.333440, step: 111, loss: 0.31579577922821045, acc: 0.8906, a

2019-03-19T10:34:15.173412, step: 165, loss: 0.1546711027622223, acc: 0.8984, auc: 0.99, precision: 0.0, recall: 0.0
2019-03-19T10:34:17.143185, step: 166, loss: 0.24293160438537598, acc: 0.8828, auc: 0.8897, precision: 0.0, recall: 0.0
2019-03-19T10:34:18.983783, step: 167, loss: 0.29229605197906494, acc: 0.8203, auc: 0.9114, precision: 0.0, recall: 0.0
2019-03-19T10:34:20.925987, step: 168, loss: 0.3532770276069641, acc: 0.8125, auc: 0.869, precision: 0.0, recall: 0.0
2019-03-19T10:34:22.816939, step: 169, loss: 0.18589088320732117, acc: 0.9062, auc: 0.9483, precision: 0.0, recall: 0.0
2019-03-19T10:34:24.589227, step: 170, loss: 0.24869762361049652, acc: 0.8281, auc: 0.9241, precision: 0.0, recall: 0.0

Evaluation:
2019-03-19T10:34:40.936206, step: 170, loss: 2.1063518789079456, acc: 0.0, auc: 0.0, precision: 0.0, recall: 0.0
Saved model checkpoint to ../model/RCNN/model/my-model-170

2019-03-19T10:34:43.756665, step: 171, loss: 0.20896661281585693, acc: 0.8984, auc: 0.9458, precisi

2019-03-19T10:37:44.456986, step: 222, loss: 0.11315456032752991, acc: 0.9531, auc: 0.9972, precision: 0.9375, recall: 0.75
2019-03-19T10:37:46.272734, step: 223, loss: 0.11637280881404877, acc: 0.9609, auc: 0.9797, precision: 0.6923, recall: 0.9
2019-03-19T10:37:48.100877, step: 224, loss: 0.1359664648771286, acc: 0.9453, auc: 0.9855, precision: 0.8462, recall: 0.6875
2019-03-19T10:37:49.903080, step: 225, loss: 0.19907160103321075, acc: 0.9453, auc: 0.9114, precision: 1.0, recall: 0.6667
2019-03-19T10:37:51.683483, step: 226, loss: 0.23923586308956146, acc: 0.8984, auc: 0.9348, precision: 0.8, recall: 0.4211
2019-03-19T10:37:53.558006, step: 227, loss: 0.17871668934822083, acc: 0.9297, auc: 0.9681, precision: 0.8125, recall: 0.6842
2019-03-19T10:37:55.395150, step: 228, loss: 0.21797972917556763, acc: 0.9297, auc: 0.9568, precision: 0.931, recall: 0.7941
2019-03-19T10:37:57.136030, step: 229, loss: 0.17356401681900024, acc: 0.9297, auc: 0.9704, precision: 0.8095, recall: 0.7727
2019-

2019-03-19T10:41:12.332027, step: 279, loss: 0.07951866090297699, acc: 0.9688, auc: 0.9876, precision: 0.8182, recall: 0.8182
2019-03-19T10:41:14.274660, step: 280, loss: 0.0652708113193512, acc: 0.9844, auc: 0.9989, precision: 1.0, recall: 0.875

Evaluation:
2019-03-19T10:41:32.764870, step: 280, loss: 1.8951290978325739, acc: 0.4114333333333333, auc: 0.0, precision: 1.0, recall: 0.4114333333333333
Saved model checkpoint to ../model/RCNN/model/my-model-280

2019-03-19T10:41:35.746388, step: 281, loss: 0.08107993006706238, acc: 0.9766, auc: 0.9967, precision: 0.9333, recall: 0.875
2019-03-19T10:41:37.630351, step: 282, loss: 0.13946808874607086, acc: 0.9531, auc: 0.9541, precision: 0.9, recall: 0.8182
2019-03-19T10:41:39.632063, step: 283, loss: 0.11505774408578873, acc: 0.9688, auc: 0.986, precision: 0.8333, recall: 0.9375
2019-03-19T10:41:41.496081, step: 284, loss: 0.1055065169930458, acc: 0.9609, auc: 0.9831, precision: 0.7778, recall: 0.7
2019-03-19T10:41:43.478769, step: 285, los

2019-03-19T10:44:56.654669, step: 335, loss: 0.0857740193605423, acc: 0.9688, auc: 0.9907, precision: 0.8333, recall: 0.8333
2019-03-19T10:44:58.660308, step: 336, loss: 0.11035880446434021, acc: 0.9688, auc: 0.9706, precision: 1.0, recall: 0.8095
2019-03-19T10:45:00.635089, step: 337, loss: 0.04193848371505737, acc: 0.9922, auc: 0.9995, precision: 1.0, recall: 0.95
2019-03-19T10:45:02.713045, step: 338, loss: 0.03923124447464943, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9524
2019-03-19T10:45:05.007937, step: 339, loss: 0.08741171658039093, acc: 0.9844, auc: 0.9837, precision: 0.9286, recall: 0.9286
2019-03-19T10:45:07.022563, step: 340, loss: 0.03764757886528969, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0

Evaluation:
2019-03-19T10:45:26.842170, step: 340, loss: 2.275788572099474, acc: 0.375, auc: 0.0, precision: 1.0, recall: 0.375
Saved model checkpoint to ../model/RCNN/model/my-model-340

2019-03-19T10:45:29.783401, step: 341, loss: 0.07365786284208298, acc: 0.9844, auc